# Tarea 6

**Envío del PR inicial:** Sábado, 3 de noviembre

**Aceptación del PR:** Viernes 9 de noviembre

**NOTA:** Esta tarea debe entregarse en equipo.

In [ ]:
include("DualNum.jl")
using TaylorSeries
using Plots

In [ ]:
Qc(x,c) = x^2+c

In [ ]:
"""
    Qcⁿ(x, args)

Esta función define el n-enésimo iterado de Qc(x,c),
donde el valor a iterar es `x` y el valor del parámetro
es `c`, donde `c = args[1]` y `n = args[2]`.
"""
function Qcⁿ(x, args)
    c, n = args[1], args[2]
    @assert n >= 0 && isa(n,Int)
    n <= 1 && return Qc(x,c)
    for it = 1:n
        x = Qc(x,c)
    end
    return x
end

# 1

- ¿Cómo se generaliza la expresión anterior para el exponente de Lyapunov en mapeos de 1 variable, a mapeos de 2 variables?


- ¿Cómo se compara el exponente de Lyapunov $\lambda(x_0)$ con $\lambda( f(x_0) )$, para el mapeo $x\mapsto f(x)$?

Al tratarse ahor a de un mapeo en dos dimensiones $f:\mathcal{R}^2\rightarrow\mathcal{R}^2$, la derivada es ahora el jacobiano $Df(\vec{x})$ que es una matriz 2x2.

Una condición inicial muy cercana a la condicion inicial $\vec{x_o}$ puede ser pensada como una vecindad de radio $\epsilon$ al redodor de $\vec{x_o}$, misma que será transformada y cambiará de forma con cada iteracion del mapeo $f(\vec{x})$.

El jacobiano del $n-ésimo$ iterado: $Df^n(\vec{x})$, tiene dos eigenvectores y dos eigenvalores. Con cada iteración habrá que hacer la propia diagonalización de la matriz $Df^n(\vec{x})$, para encontrar los eigenvalores y eigenvectores. Tales vectores serán las direcciones en las que se alarga o se contrae la vecindad (circulo,elipse,o de cualquier otra forma) del punto $f^n(x_o)$.

Es necesario entonces que se definan $n$ exponentes de Lyapunov, si se trata un mapeo de $n$ dimensiones, puesto que una vecindad tendrá $n$ direcciones en las que se alarga o contrae (eigenvectores), en el caso de 2 variables, existirán dos exponentes de Lyapunov $\lambda_1(\vec{x})$ y $\lambda_2(\vec{x})$, que estarán dados por:
$$
\lambda_k(\vec{x_o}) = \lim_{n\rightarrow\infty}\ln\left(\prod_{i=1}^n| y_k^i |\right)
$$
donde $y_k^i$ es la longitud de la $k-ésimo$ dirección o eigenvector de la vecindad para el iterado $i-ésimo$ de la condición inicial $\vec{x_o}$.

La expresión para el exponente de Lyapunov con condición inicial correspondiente a la pripera aplicación del maepo $f$ en $\vec{x_o}$, sería:
$$
\lambda_k(f(\vec{x_o})) = \lim_{n\rightarrow\infty}\ln\left(\prod_{i=2}^n| y_k^i |\right)
$$
que como se trata de la misma órbita y en principio de debe tomar el límite cuando $n\rightarrow\infty$, resultaría ser el mismo número. Mas adelante se verá este aspecto para el mapeo cuadrático.

# 2

Obtengan, para la familia de mapeos cuadrática $Q_c(x)$, el diagrama que muestra el exponente de Lyapunov en términos de $c$, para $c\in[-2,2]$.

- ¿Qué particularidad ocurre (en términos del exponente de Lyapunov) para los valores de $c$ donde hay bifurcaciones?

- ¿Qué particularidad ocurre (en términos del exponente de Lyapunov) cuando tenemos *ciclos superestables*, es decir, cuando $x=0$ es parte de un ciclo periódico?

In [ ]:
"""
    iterados(f,nout,crange)
Devuelve una matriz cuyas filas corresponden al n-ésimo iterado de la función `f`.
Las columnas corresponden a un mismo parámetro `c` de la función `f`.
"""
function iterados(f,x0,nout,crange)
    
    ff = Array{Float64,2}(undef, nout, length(crange))
    
    
    
    for ic in eachindex(crange)
        x = x0
        cc = crange[ic]
        #@inbounds ff[1,ic] = x0
        for it = 1:nout
            x = f(x, cc)
            @inbounds ff[it, ic] = x
        end

    end
    
    return ff

end

In [ ]:
function lambda(f, x0,nout,crange)
    
    ff = iterados(f,x0,nout,crange)
    
    L = Array{Float64,1}(undef, length(crange))
    
    for ic in eachindex(crange)
        sum = 0.0
        for it in 1:nout
            sum = sum + log(abs(f(DualNum.dual(ff[it,ic]),crange[ic]).x´))
        end
        lambda = sum/nout
        @inbounds L[ic] = lambda
    end
    
    return L
end

A continuación se puede ver la diferencia entre los exponentes de Lyapunov para una condición inicial ($x_o=0.25$) usando el mapeo cuadrático con $c=-1.5$ y el respecivo exponente para la condición inicial $Q_{-1.5}(0.25)$.
El error es del orden de $10^-{10}$, lo cual nos dice que deberpia ser el mismo, no lo es pues computacionalmente $n\rightarrow\infty$ solo se hace para una $n$ muy grande.

In [ ]:
a = lambda(Qc,0.25,1000000,-1.5)
b = lambda(Qc,Qcⁿ(0.25,(-1.5,2)),1000000,-1.5)

println(abs(a[1]-b[1]))

In [ ]:
E_L = lambda(Qc,0.1,1000000,-2.0:0.02:1.0)
plot(-2.0:0.02:1.0, E_L, legend = false,xlabel="c", ylabel="Exp. de Lyapunov",title="Xo=0.1")

In [ ]:
E_L1 = lambda(Qc,-1.4,100000,-2.0:0.02:1.0)
plot(-2.0:0.02:1.0, E_L1,legend=false, xlabel="c", ylabel="Exp. de Lyapunov",title="Xo=-1.4")

Se observa que para $c$ tal que hay bifurcación, el exponente de Lyapunov $\lambda(x_o)$ es cero o muy cercano a cero.

Mientras que cuando se tiene $c$ para un ciclo super estable, el exponente de Lyapunov $\lambda(x_o)\rightarrow-\infty$, lo cuál hace sentido, pues así dos condiciones iniciales separadas por una distancia $\epsilon$, convergen muy rápidamente a $x=0$ pues es un ciclo super estable.

# 3

Rehacer los cálculos de los exponentes de Feigenbaum,
\begin{eqnarray*}
f_n &=& \frac{C_n-C_{n+1}}{C_{n+1}-C_{n+2}},\\
a_n &=& \frac{d_n}{d_{n+1}},
\end{eqnarray*}

usando los valores $C_n$ donde se tienen ciclos superestables de periodo $2^n$. (Este ejercicio es muy parecido a los ejercicios 1 y 3 de la `Tarea4`.) 

Nota: Es importante dar un punto suficientemente bien aproximado a $C_n$ para que el método de Newton converja bien.

In [ ]:
function roots_newton(f, x0)
    t = Taylor1(eltype(x0), 1)
    for ind = 1:100
       fT = f(x0+t)
       x0 = x0 - fT[0]/fT[1]
    end
    x0
end

In [ ]:
C0 = 0.0
C1 = -1.0

Q4c = c -> Qcⁿ(0.0, (c, 4))
C2 = roots_newton(Q4c, -1.4)


Q8c = c -> Qcⁿ(0.0, (c, 8))
C3 = roots_newton(Q8c, -1.4)

Q16c = c -> Qcⁿ(0.0, (c, 16))
C4 = roots_newton(Q16c, -1.4)

Q32c = c -> Qcⁿ(0.0, (c, 32))
C5 = roots_newton(Q32c, -1.4)

Q64c = c -> Qcⁿ(0.0, (c, 64))
C6 = roots_newton(Q64c, -1.401)

Q128c = c -> Qcⁿ(0.0, (c, 128))
C7 = roots_newton(Q128c, -1.4011)

Q256c = c -> Qcⁿ(0.0, (c, 256))
C8 = roots_newton(Q256c, -1.40115)

In [ ]:
Cn=[C0,C1,C2,C3,C4,C5,C6,C7,C8]
fn(c1,c2,c3) = (c1-c2)/(c2-c3)

In [ ]:
Fn=[]
for i in 1:7
    push!(Fn,fn(Cn[i],Cn[i+1],Cn[i+2]))
end
Fn

Para determinar la sucesión $a_n = d_n/d_{n+1}$. Ya que tenemos la sucesión $C_n$ determinaremos los puntos de periodo $2^p$ para tales $C_n$.

In [ ]:
"""
    ciclosestables!(xx, f, cc, nit, nout)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0, cc)` tenga sentido. La idea es que los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`.
"""
function ciclosestables!(xx, f, cc, nit, nout)
    @assert (nit > 0) && (nout > 0)
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    nothing
end

In [ ]:
"""
    diag_bifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diag_bifurc(f, nit, nout, crange)
#     xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(undef, nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(view(ff, :, ic), f, c, nit, nout)
#         ff[:,ic] = xx
    end
    
    return ff
end

In [ ]:
D = []
for i in 1:9

    A = diag_bifurc(Qc, 1000000, 2^(i-1), Cn[i])          #Se usarán los valores de c determinados en el inciso anterior.
    
    absA = []                    #Se tomarán los valores absolutos del vector A, para poder nombrar al más cercano a cero
                                 #como el mínimo de éste vector absA
    for l in 1:length(A)
        push!( absA , abs(A[l]))
    end
    
    min = minimum(absA)
    s = 0
    
    for r in 1:length(absA)      #Se guarda el ínidce tal que el la entrada de A es la más cercana a cero.
        if absA[r] == min
            s = r
        end
    end


    D1 = []
    
    if i == 1
        push!(D1,0.0)
    end
    
    for m in 1:length(A)        #Se realiza la resta de los demás puntos fijos de A, con el más cercano a cero y se guardan en D1
        if m != s
            push!(D1, abs(min-A[m]))
        end
    end
    
    push!(D, minimum(D1))       #Por último en el vector D, se guarda el menor de las restas anteriores.
    
end

In [ ]:
dn(d1,d2) = d1/d2
α = []
for i in 1:8
    push!(α, dn(D[i],D[i+1]))
end
α

# 4

Comparar gráficamente los mapeos $Q_{c}(x)$ y $Q_{c^\prime}^2(x)$ para $c=1/4, 0, -3/4, -1, -2, -2.2$ y $c^\prime=-3/4,-1,-5/4,-1.3, -1.546\dots, -1.65$. 


In [ ]:
xrange = -3:1/128:3
p1 = plot(xrange, x->Qc(x,1/4),legend=false,title="c=1/4",xlabel="x",ylabel="Qc(x)")
plot!(xrange,x->x)
p2 = plot(xrange, x->Qc(x,0.0),legend=false,title="c=0.0",xlabel="x",ylabel="Qc(x)")
plot!(xrange, x->x)
p3 = plot(xrange, x->Qc(x,-3/4),legend=false,title="c=-3/4",xlabel="x",ylabel="Qc(x)")
plot!(xrange, x->x)
p4 = plot(xrange, x->Qc(x,-1.0),legend=false,title="c=-1",xlabel="x",ylabel="Qc(x)")
plot!(xrange, x->x)
p5 = plot(xrange, x->Qc(x,-2.0),legend=false,title="c=-2",xlabel="x",ylabel="Qc(x)")
plot!(xrange, x->x)
p6 = plot(xrange, x->Qc(x,-2.2),legend=false,title="c=-2.2",xlabel="x",ylabel="Qc(x)")
plot!(xrange, x->x)

In [ ]:
x = -2:1/128:2
P1 = plot(x, x->Qcⁿ(x,(-3/4,2)),legend=false,title="c=-3/4",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)
P2 = plot(x, x->Qcⁿ(x,(-1.0,2)),legend=false,title="c=-1",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)
P3 = plot(x, x->Qcⁿ(x,(-5/4,2)),legend=false,title="c=-1.25",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)
P4 = plot(x, x->Qcⁿ(x,(-1.3107026413368328,2)),legend=false,title="c=-1.3",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)
P5 = plot(x, x->Qcⁿ(x,(-1.546,2)),legend=false,title="c=-1.546",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)
P6 = plot(x, x->Qcⁿ(x,(-1.65,2)),legend=false,title="c=-1.65",xlabel="x",ylabel="Qc(x)")
plot!(x, x->x)

In [ ]:
plot(p1,P1)

In [ ]:
plot(p2,P2)

In [ ]:
plot(p3,P3)

In [ ]:
plot(p4,P4)

In [ ]:
plot(p5,P5)

In [ ]:
plot(p6,P6)

En las gráficas anteriores se observa como ciertas secciones de las gráficas de la izquierda son muy similares las de la derecha, muy cerca de intersectar a la función identidad.

# 5

Estudiar la dinámica para $c=-2.2$

- ¿Cómo se caracteriza el valor al que tiende *casi* cualquier condición inicial en el
intervalo $I=[-p_+,p_+]$ (donde $p_+$ es el valor positivo tal que $Q_c(p_+)=p_+$), después de muchos iterados? (El intervalo $I$ es donde "toda la dinámica interesante ocurre".)

- Caractericen los subconjuntos de $I$ tales que, después de $n=1,2,3,\dots$ iterados de $Q_c(x)$, su intersección con $I$ es vacía.

- Caracterizen al complemento del conjunto de los casos anteriores, es decir, cuando los iterados de la condición inicial **no** llegan a donde *casi* cualquier condición inicial termina. ¿Cómo se conoce a este conjunto?

In [ ]:
p = 0.5+sqrt(9.8)/2

In [ ]:
x=-p:1/4096:p
pm = []
pM = []
X = []
for xo in x
    push!(X,Qcⁿ(xo,(-2.2,5)))
    push!(pm,-p)
    push!(pM,p)
end
plot(x,X,title="c=-2.2",label="Q^5(x)",ylims=(-2.5,2.5),xlabel="x",ylabel="Qc^5(x)")
plot!(x,pm,label="-p+")
plot!(x,pM,label="p+")

Vemos que tan solo para el quinto iterado, hay muchos subintervalos de $I = [-p_+,p_+]$ tales que el mapeo se sale del mismo intervalo $I$ y divergen hasta infinito si $n\rightarrow\infty$. Abajo graficamos los puntos iniciales $x_o$ para los cuáles un iterado muy grande no se sale del intervalo $I$.

In [ ]:
x=-p:1/8192:p
X1 = []
for xo in x
    x1 = Qcⁿ(xo,(-2.2,20))
    if abs(x1) < p
        push!(X1,0)
    else
        push!(X1,1)
    end
end
scatter(x,X1,ylims=(-0.5,0.5), markersize=0.1,color=:red,xlabel="x",title="xo t.q. Qc^20 pertenece a I,con c=-2.2", legend=false)

In [ ]:
x=-p:1/8192:p
X1 = []
for xo in x
    x1 = Qcⁿ(xo,(-2.2,30))
    if abs(x1) < p
        push!(X1,0)
    else
        push!(X1,1)
    end
end
scatter(x,X1,ylims=(-0.5,0.5), markersize=0.1,color=:red,xlabel="x",title="xo t.q. Qc^30 pertenece a I,conc=-2.2", legend=false)

Se observa que a medida que $n$ crece, los intervalos para los cuáles $Q_c^n(x)$ no sale del intervalo $I=[-p_+,p_+]$, se hacen cada vez de menor longitud porque se van partiendo en subintervalos de menor longitud.

In [ ]:
x=-p:0.000005:p
X1 = []
for xo in x
    x1 = Qcⁿ(xo,(-2.2,60))
    if abs(x1) < p
        push!(X1,0)
    else
        push!(X1,1)
    end
end
scatter(x,X1,ylims=(-0.5,0.5), markersize=0.1,color=:red,xlabel="x",title="xo t.q. Qc^30 pertenece a I,conc=-2.2", legend=false)

# 6

- Usando el análisis gráfico para un valor de $c$ *ligeramente* anterior al punto donde se observa la ventana de periodo 3 para la familia $Q_c(x)$, y uno en la ventana de periodo 3, concluyan el tipo de bifurcación que se observa.

- Comparen la dinámica para ambos valores de $c$ del inciso anterior. ¿Cómo es el exponente de Lyapunov en cada caso?

- Encuentren el valor de $c$ donde se da el ciclo superestable de periodo 3. (Nuevamente, usen el método de Newton con una buena semilla inicial para $c$.) Demuestren que para este valor de $c$ el mapeo $Q_c^3(x)$ tiene *ocho* puntos fijos, donde dos de ellos son puntos fijos de $Q_c(x)$, y los otros 6 forman dos secuencias periódicas, de periodo 3 en $Q_c(x)$, una de ellas estable y la otra inestable.

(Los siguientes incisos usan el valor de $c$ encontrado en el inciso anterior.)

- Denotando al ciclo inestable de periodo 3 como $\alpha\mapsto \beta\mapsto\gamma$, con $\gamma<\beta<\alpha$, uno puede asociar a cada uno de estos valores un valor $\hat\alpha, \hat\beta, \hat\gamma$, tales que $Q_c^3(\hat\alpha)=\alpha$, $Q_c^3(\hat\beta)=\beta$, $Q_c^3(\hat\gamma)=\gamma$. ¿Qué podemos concluir de un punto que cae en cualquiera de los intervalos $[\alpha,\hat\alpha]$, $[\beta,\hat\beta]$ y $[\hat\gamma,\gamma]$, en términos de los iterados sucesivos?

- Definiendo los intervalos $I_0=[\hat\beta, \alpha]$, $I_1=[\gamma,\beta]$, mostrar que bajo la acción de $Q_c(x)$ se cumple: (i) $Q_c(I_0) \supset I_1$, y (ii) $Q_c(I_1) \supset I_0\cup I_1$. ¿Qué propiedad *importante* de $Q_c(x)$ se utiliza aquí?

(Los intervalos $I_0$ e $I_1$, y estas propiedades, son básicos en la demostración del teorema "periodo tres implica caos".)

In [ ]:
function bifurcación2(f,c,xo,nin)
    C = []
    X = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    x1 = xo
    
    for a in c
        for i in 1:nin
            x1 = f(x1,a)
        end
        for j in 1:32
            x1 = f(x1,a)
            push!(X[j],x1)
        end
        push!(C,a)   
    end
    
    p1 = scatter(C,X[1], xaxis = ("c", (-1.76,-1.73)), legend=false,yaxis = ("X"), title="Bifurcación x^2+c", color=:blue)
    for i in 2:32
        scatter!(C,X[i], color=:blue)
    end
    
    return p1
end

In [ ]:
bifurcación2((x,a)->x^2+a,-1.76:0.0005:-1.745,0.0,1000)

Se observa que hay una rara bifurcación en el punto $c=-1.75$, donde el periodo infinito de las órbitas para $c>-1.75$ se transforma repentinamente en periodo $3$.

In [ ]:
E_L = lambda(Qc,0.0,1000000,-1.757:0.0001:-1.745)
plot(-1.757:0.0001:-1.745, E_L, legend = false,xlabel="c", ylabel="Exp. de Lyapunov",title="Xo=0.1")

El exponente de Lyapunov es mayor a cero para $c>-1.75$, donde justamente se tienen los ciclos de periodo $2^n$ (muy grandes), cuyas órbitas con condiciones iniciales cercanas, no convergen a valores cercanos, hay caos.

Justo en el valor $c=-1.75$, el exponente de Lyapunov se anula pasando a ser negativo si $c<-1.75$, lo cuál inidica que ahora las órbitas tenderán a puntos fijos de periodo finito, órbitas estables. Si continuamos barriendo $c$ se llega a un punto ($c\approx-1.755$) en el cuál el exponente de Lyapunov diverge a $-\infty$, lo que indica que tal $c$ genera un ciclo super estable, y es de periodo $3$, como o muestra la gráfica de la bifurcación en $c=-1.75$.

El valor se aproxima con el método de Neton aplicado al mapeo $Q^3(0,c)$ empezando con $c=-1.755$.

In [ ]:
Q3c = c->Qcⁿ(0.0,(c,3))
C3 = roots_newton(Q3c, -1.755)

In [ ]:
x = -2:1/128:2

plot(x,x->Qcⁿ(x,(C3,3)),label="Q^3(x)",xlabel="x",ylabel="Q^n(x)")
plot!(x,x->Qcⁿ(x,(C3,1)),label="Q(x)")
plot!(x,x->x,label="x")

Vemos que el mapeo $Q_c^3(x)$ con $c=-1.75487$, tiene $8$ puntos fijos, donde dos de ellos son también puntos fijos de $Q_c(x)$. Son $x\approx-1,2$.

Haciendo zoom en los intervalos donde se encuentran los puntos fijos y utilizando el método de Newton se encuentran tales puntos fijos del mapeo $Q_c^3(x)$.

In [ ]:
x = -1.775:1/512:-1.735

plot(x,x->Qcⁿ(x,(C3,3)),label="Q^3(x)",xlabel="x",ylabel="Q^n(x)")
plot!(x,x->x,label="x")

In [ ]:
x = -0.15:1/512:0.15

plot(x,x->Qcⁿ(x,(C3,3)),label="Q^3(x)",xlabel="x",ylabel="Q^n(x)")
plot!(x,x->x,label="x")

In [ ]:
x = 1.277:1/512:1.37

plot(x,x->Qcⁿ(x,(C3,3)),label="Q^3(x)",xlabel="x",ylabel="Q^n(x)")
plot!(x,x->x,label="x")

In [ ]:
Q3x = x->Qcⁿ(x,(C3,3))-x
r1 = roots_newton(Q3x, -1.75)
r2 = roots_newton(Q3x, -1.74)
r3 = roots_newton(Q3x, -1)
r4 = roots_newton(Q3x, -0.1)
r5 = roots_newton(Q3x, 0.0)
r6 = roots_newton(Q3x, 1.28)
r7 = roots_newton(Q3x, 1.32)
r8 = roots_newton(Q3x, 2)

r=[r1,r2,r3,r4,r5,r6,r7,r8]

println("Los puntos fijos de Qc^3(x) son:")
for i in 1:8
    println("r$i = ", r[i])
end

donde r1,r5 y r7 son los puntos que perteneces a la órbita estable de periodo 3.

r2, r4 y r6 perteneces a otra órbita de periodo 3 pero inestable.

r3 y r8 son los puntos fijos que también lo son para el mapeo $Q_c(x)$.

Observando las tres gráficas anteriores nos damos cuenta de que podemos definir intervalos para los cuáles si la condición inicial $x_o$ cae dentro de tales intervalos, ocurrira algo particular con los iterados siguientes.

Definamos los siguientes puntos que ayudarán a definir tales intervalos.

In [ ]:
α = r7 + (r7-r6)
β = r5 + (r5-r4)
γ = r1 + (r1-r2)

Ahora veamos qué pasa con muchas condiciones iniciales dentro de los intervalos siguientes:

$[γ,r_1]$ , $[r_5,β]$ , $[r_7,α]$

In [ ]:
x = γ:0.001:r1
plot(x,x->Qcⁿ(x,(C3,8)),xlabel="x",ylabel="Q^27(x)",legend=false)

In [ ]:
x = γ:0.001:r1
plot(x,x->Qcⁿ(x,(C3,27)),xlabel="x",ylabel="Q^27(x)",legend=false)

Si una condición inicial cae dentro de uno de los intervalos $[γ,r_1]$ , $[r_5,β]$ , $[r_7,α]$, la órbita va saltado de un intervalo al siguiente, al siguiente y regresa al inicial un número indefinido de veces, quedándose atrapada dentro de éstos tres intervalos y además convergiendo a los puntos fijos estables (órbita de periodo 3).

Ahora definamos los intervalos $I_0=[β,r_7]$ y $I_1=[r_1,r_5]$

In [ ]:
Io = β:1/512:r7
I1 = r1:1/512:r5

In [ ]:
plot(Io,x->Qcⁿ(x,(C3,1)),xlabel="x",ylabel="Q^n(x)",label="Qc(Io)")
plot!(I1,x->x,xlabel="x",label="I1")

Vemos que el intervalo $I_1$ está contenido dentro de la primera aplicación con $Q_c(x)$ del intervalo $I_0$.

Y en la gráfica de abjo se observa que la unión $I_oUI_1$ está contenida en la imagen de $I_1$ bajo $Q_c(x)$, $(Q_c(I_1))$.

Todo lo anterior con $c=-1.7548$.

In [ ]:
plot(I1,x->Qcⁿ(x,(C3,1)),xlabel="x",ylabel="Q^n(x)",label="Qc(I1)")
plot!(I1,x->x,xlabel="x",label="I1")
plot!(Io,x->x,xlabel="x",label="Io")